In [17]:
import json


In [18]:
with open('documents-llm.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [19]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [20]:
documents[0]

{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [28]:
from elasticsearch import Elasticsearch


In [29]:
es_client = Elasticsearch('http://localhost:9200') 


In [30]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [31]:
documents[0]


{'text': 'Yes, but if you want to receive a certificate, you need to submit your project while we’re still accepting submissions.',
 'section': 'General course-related questions',
 'question': 'I just discovered the course. Can I still join?',
 'course': 'llm-zoomcamp'}

In [53]:
from tqdm.auto import tqdm


In [54]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/86 [00:00<?, ?it/s]

In [58]:
query="how to get access to saturn cloud"


In [59]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "llm-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [60]:
search_results = elastic_search(query)
search_results

[{'text': 'Please see the General section or use CTRL+F to search this doc.',
  'section': 'Module 2: Open-Source LLMs',
  'question': 'Saturn Cloud issues',
  'course': 'llm-zoomcamp'},
 {'text': 'Issue: I get the notice that due to traffic, I’m on a waitlist for new signups.\nAnswer: There was a form to submit our emails to, so Alexey can send it in bulk. If you missed that deadline, just sign up manually (or via request tech demo link) and use the chat to request for free hours for “llm zoomcamp”\nIssue: I’m a pre-existing user from a different zoomcamp and I’m not awarded the free hours even though I’ve submitted my email in the form.\nAnswer: Just request it via their chat, after you’ve logged in using your pre-existing account, citing “llm zoomcamp” .',
  'section': 'General course-related questions',
  'question': 'SaturnCloud - How do I get access?',
  'course': 'llm-zoomcamp'},
 {'text': 'Manually set the token as below:\naccess_token = <your_token>\nmodel  = AutoModelForCausa

In [ ]:
## Libraries Required
%pip install langchain-huggingface --quiet
## For API Calls
%pip install huggingface_hub --quiet
%pip install transformers --quiet
%pip install accelerate --quiet
%pip install  bitsandbytes --quiet
%pip install langchain --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note

In [ ]:
import os 
%pip install dotenv 
from dotenv import load_dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
load_dotenv()

True

In [ ]:
key=os.environ["HUGGINGFACEHUB_API_TOKEN"]
os.environ["HUGGINGFACEHUB_API_TOKEN"]=key

In [ ]:
import os
from huggingface_hub import InferenceClient


def llm(prompt):
    client = InferenceClient(
        provider="groq",
        api_key=key,
    )

    completion = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
    )

    return completion.choices[0].message.content

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [61]:

query="how to get access to saturn cloud"
rag(query)

'**How to get access to Saturn\u202fCloud for the LLM Zoomcamp**\n\n1. **Submit your email**  \n   - There was a shared form where you could submit your email address. Alexey used that list to grant access in bulk.  \n   - **If you missed that deadline**, simply sign up on the Saturn\u202fCloud site (or use the “request tech demo” link) and then **use the Saturn\u202fCloud chat** to ask for free hours, mentioning **“llm zoomcamp.”**\n\n2. **If you already have a Saturn\u202fCloud account from a previous Zoomcamp**  \n   - Log in with your existing account.  \n   - Open the Saturn\u202fCloud chat and request the free hours, again citing **“llm zoomcamp.”**  \n\nThe key steps are:\u202fsubmit your email (or sign up manually), then request the free “LLM Zoomcamp” hours via the chat (or have Alexey add you if you’re on the original email list).'